<a href="https://colab.research.google.com/github/honourjesus20/Geosemantics/blob/main/Geosematics_map_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import json
from spacy.tokens import DocBin

In [ ]:
# Load your trained spacy model
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Load your data from a JSON file
with open('/content/annotations (1).json') as f:
    data = json.load(f)

In [ ]:
# Initialize DocBin to store the processed documents
db = DocBin()

In [ ]:
for item in data['annotations']:
    # Skip None entries
    if item is None:
        continue

    text, annot = item

    # Ensure the annotation has the "entities" key
    if "entities" not in annot:
        print("Annotation missing 'entities' key. Skipping...")
        continue

    doc = nlp.make_doc(text)
    ents = []

    for start, end, label in annot["entities"]:
        # Only process location entities
        if label == 'LOCATION':
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)

    doc.ents = ents
    db.add(doc)

# Save the docbin object
db.to_disk("./location_data.spacy")

In [ ]:
# Load the DocBin file
db = DocBin().from_disk("./location_data.spacy")

# Iterate through the documents in the DocBin
for doc in db.get_docs(nlp.vocab):
    # Iterate through the entities in each document
    for ent in doc.ents:
        # Print the text and label of each entity
        print(f"{ent.text} ({ent.label_})")

Lagos (LOCATION)
Lagos (LOCATION)
Nigeria (LOCATION)
Nigeria (LOCATION)
Balogun Market (LOCATION)
Mile 12 Market (LOCATION)
Lagos (LOCATION)
Lekki Toll Gate (LOCATION)
Balogun Market (LOCATION)
Lagos Island. (LOCATION)
Victoria Island (LOCATION)
Balogun (LOCATION)
Mile 12 Market (LOCATION)
Lagos (LOCATION)
Tarkwa Bay (LOCATION)
National Theatre (LOCATION)
Lagos (LOCATION)
Ikeja (LOCATION)
Ikoyi (LOCATION)
Lagos (LOCATION)
Lagos (LOCATION)
West Africa (LOCATION)
Nigeria. (LOCATION)
Lagos (LOCATION)
Nigeria’s (LOCATION)
Lekki Toll Gate (LOCATION)
Balogun Market (LOCATION)
Lagos Island (LOCATION)
Lagos (LOCATION)
Eko Bridge (LOCATION)
Lagos. (LOCATION)
mainland (LOCATION)
Mile 12 Market (LOCATION)
Lagos. (LOCATION)
Tarkwa Bay (LOCATION)
Lagos Harbor (LOCATION)
National Theatre (LOCATION)
Lagos’ (LOCATION)
Ikeja (LOCATION)
Apapa (LOCATION)
Lagos (LOCATION)
Lagos (LOCATION)
Lagos (LOCATION)
Lagos. (LOCATION)


In [ ]:
from geopy.geocoders import GeoNames

In [ ]:
import folium

# Initialize an empty map
# You may want to set the initial location and zoom level based on your data
map = folium.Map(location=[0, 0], zoom_start=2)

# Initialize the geolocator with your GeoNames username
geolocator = GeoNames(username="boibash")

# Iterate through the documents in the DocBin
for doc in db.get_docs(nlp.vocab):
    # Iterate through the entities in each document
    for ent in doc.ents:
        # Only process LOCATION entities
        if ent.label_ == 'LOCATION':
            location = geolocator.geocode(ent.text)
            if location:
                print(f"{ent.text}: Latitude = {location.latitude}, Longitude = {location.longitude}")
                # Create a marker for each location
                folium.Marker(
                    location=[location.latitude, location.longitude],
                    popup=ent.text
                ).add_to(map)
            else:
                print(f"Location not found for {ent.text}")

# Save the map to an HTML file
map.save('locations_map.html')

Lagos: Latitude = 6.45407, Longitude = 3.39467
Lagos: Latitude = 6.45407, Longitude = 3.39467
Nigeria: Latitude = 10.0, Longitude = 8.0
Nigeria: Latitude = 10.0, Longitude = 8.0
Location not found for Balogun Market
Location not found for Mile 12 Market
Lagos: Latitude = 6.45407, Longitude = 3.39467
Location not found for Lekki Toll Gate
Location not found for Balogun Market
Lagos Island.: Latitude = 6.46113, Longitude = 3.39037
Victoria Island: Latitude = 22.2875, Longitude = 114.14417
Balogun: Latitude = 8.46569, Longitude = 4.82754
Location not found for Mile 12 Market
Lagos: Latitude = 6.45407, Longitude = 3.39467
Tarkwa Bay: Latitude = 6.42995, Longitude = 3.40636
National Theatre: Latitude = 38.89668, Longitude = -77.02578
Lagos: Latitude = 6.45407, Longitude = 3.39467
Ikeja: Latitude = 6.59651, Longitude = 3.34205
Ikoyi: Latitude = 6.46034, Longitude = 3.43507
Lagos: Latitude = 6.45407, Longitude = 3.39467
Lagos: Latitude = 6.45407, Longitude = 3.39467
West Africa: Latitude = 13